In [ ]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Kruskal + TSP


Prim's algorithm does a graph search while computing the MST.

Can we just greedily add edges in increasing order of weight?

**Almost**: need to avoid cycles and ensure we "span" the graph

![figures/greedy-fail.png](figures/greedy-fail.png)


<br>

### Kruskal's Algorithm

“Perform the following step as many times as possible: Among the edges of $G$ not yet chosen, choose the shortest edge which does not form any loops with those edges already chosen.” [Kruskal, 1956]

**Why is this correct?**

Recall the **Light-edge property**

Let $G = (V,E,w)$ be a connected undirected, weighted graph with distinct edge weights. 

For any cut of $G$, the minimum weight edge that crosses the cut is contained in the minimum spanning tree of $G$.

<center>
    <img src="figures/cut.jpg"/>
</center>

At each step, Kruskal picks a **light edge** to connect to components of the graph $U$ and $V$ (i.e., picks an edge $e$ where $w(e) < w(e')$).

<br>
How do we know the tree spans all the nodes?

<br><br>
The only edges we've skipped are those that create a cycle, so the remaining edges must form a tree.

### implementing Kruskal's algorithm

"choose the shortest edge which does not form any loops with those edges already chosen"

How do we implement this?

For an edge $(u,v)$, we must check if $u$ and $v$ are in the same connected component, based on the edges added so far.

How can we do this?

We can run BFS or DFS starting at $u$ or $v$, but this would be expensive: $O(|V| + |E|)$ work at each iteration.

<br>

If we think of each connected component as a set of nodes, we need an efficient way of:

- checking which set $u$ and $v$ are in
- determining if these two sets are equal
- if they are not equal, then we need to take their union


<br>
To make checking set equality fast, we will assign a **representative** node in each set.

E.g., suppose we have two sets $\mathbf{S} = \{S_1, S_2\}$ where:

$S_1 = \{\mathbf{a},b,c\}~~~~ S_2 = \{\mathbf{s}, d, e\}$

We can (arbitrarily) assign the representative of $S_1$ to be $a$, and the representative of $S_2$ to be $s$.

$r(S_1) = a ~~~ r(S_2) = s$

<br>

If $S_u$ is the set containing $u$ and $S_v$ is the set containing $v$, then we can check if $u$ and $v$ are in the same set by checking if $r(S_u) == r(S_v)$





## Disjoint-set data type

We have described a **disjoint-set** abstract data type (aka **union-find**). This type supports three operations:

1. `make_set(u)`: create a new set containing the single element $u$
  - $u$ will be the representative of this set
  
  
2. `find_set(u)`: returns the representative of the set containing $u$:  $r(S_u)$


3. `union(u,v)`: replace $S_u$ and $S_v$ with $S_u \cup S_v$ in the collection of sets $\mathbf{S}$

What data structures can we use to represent each set?

### Disjoint-set with double linked lists

<center>
    <img src="figures/delist.png"/>
</center>

[source](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-046j-design-and-analysis-of-algorithms-spring-2012/lecture-notes/MIT6_046JS12_lec16.pdf)


Each set is a doubly-linked list. The first node is the representative.

How can we perform the three operations and what is the work?

1. `make_set(u)`: create a new set containing the single element $u$
  - $u$ will be the representative of this set
  
  
2. `find_set(u)`: returns the representative of the set containing $u$:  $r(S_u)$


3. `union(u,v)`: replace $S_u$ and $S_v$ with $S_u \cup S_v$ in the collection of sets $\mathbf{S}$

<br><br>

1. `make_set(u)`: create a singleton list in $O(1)$ time. $u$ is the representative.
  
  
2. `find_set(u)`: walk left from $u$ until the first element is found. $O(n)$


3. `union(u,v)`: Walk to the tail of $S_u$ and connect it to the head of $S_v$. $O(n)$

<br><br>

That's a lot of needless walking. Can I just store the representative directly at each node?

<center>
    <img src="figures/delist2.png"/>
</center>

This would make `find_set(u)` in $O(1)$, but union is still $O(n)$. Why?

## Disjoint-set with balanced forests

<center>
    <img src="figures/forest.png"/>
</center>

Each set is a balanced tree, where the root is the representative.

Assuming we represent a tree node with a pointer to its parent, what is the work of `find_set(u)` (to find the representative of $S_u$?

$O(\log n)$, assuming a balanced tree, to walk from a node to its root.

How about `union(u,v)`?


<center>
    <img src="figures/merge.png" width=50%/>
</center>

- find representative of $u ~~~~ O(\log n)$
- find representative of $v ~~~~ O(\log n)$
- link root of one tree to the root of another

<br>

But, how do we ensure tree stays balanced to keep $O(\log n)$ work for `find_set(u)`?


Recall [Leftist Heaps](https://nbviewer.jupyter.org/github/tulane-cmps2200/slides/blob/master/module-06-greedy/greedy-02.ipynb?flush_cache=True) from Greedy module!

<center>
    <img src="../module-06-greedy/meld_schematic.jpg" width="70%">
</center>

similar idea here:

<center>
    <img src="figures/rank.png" width="70%">
</center>

Add "shorter" tree to the "taller" tree.
- store the "rank" of each tree as its depth

<br>
Thus, if $height(S_v) < height(S_u)$, then the height the union of $S_u \cup S_v$ is 

$\max \{ height(S_u), height(S_v)+1\}$

<br>
Using similar arguments as in leftist heaps, we can ensure that height of any tree is $O(\log n)$

### Kruskal's Algorithm

0. Initialize tree $T \leftarrow \emptyset$


1. For each $v \in V$, run `make_set(v)`


2. Sort edges in increasing order of weight


3. For each edge $e=(u,v)$ in sorted set:
  - if `find_set(u)` $\ne$ `find_set(v)`:
    - $T \leftarrow T \cup \{(u,v)\}$
    - `union(u,v)`


### Kruskal's Algorithm - work

0. Initialize tree $T \leftarrow \emptyset~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~O(1)$


1. For each $v \in V$, run `make_set(v)` $~~~~~O(|V|)$


2. Sort edges in increasing order of weight $~~O(|E|\lg |E|)$


3. For each edge $e=(u,v)$ in sorted set: $~~O(|E|\lg |V|)$
  - if `find_set(u)` $\ne$ `find_set(v)`: $~~O(\lg |V|)$
    - $T \leftarrow T \cup \{(u,v)\}$
    - `union(u,v)`
    
    
Thus, total work is $O(|E|\lg |E|)$

Since we have $E \in O(|V|^2)$, this is equivalent to:

$O(|E|\lg |V|^2) \equiv O(2|E| \log |V|) \equiv O(|E| \log |V|)$


## Traveling Salesperson Problem

Consider a slight variant of the MST problem:

Given a graph $G=(V,E)$, find a tour that visits each node exactly once and then returns to the origin node.
 - every node is visited
 - no edges are repeated

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/GLPK_solution_of_a_travelling_salesman_problem.svg/480px-GLPK_solution_of_a_travelling_salesman_problem.svg.png"/>
</center>

Often, we assume the graph is *complete* (fully connected) and edge weights are distance between each city.

<br>

How does this differ from the MST problem?

- TSP solution has one more edge than MST solution (graph instead of a tree)


- Therefore, weight(TSP solution) < weight(MST solution)


Can we use MST solution to provide a lower bound on the TSP solution?

### Euclidean TSP

Variant of TSP where triangle inequality holds:

$w(u,v) + w(v,w) \ge w(u,w)$

where all weights are non-negative.

Consider a MST solution for the graph:

<center>
    <img src="figures/tsp1.jpg"/>
</center>

<br>

How could we convert this tree into a tour for TSP?

<br><br>


We need to determine an order to visit the nodes in the MST solution.

Let's try depth-first search:

<center>
    <img src="figures/tsp2.jpg"/>
</center>

This is called the **Euler tour** of the tree:

 - a cycle in a graph that visits every edge exactly once.
 - Since $T$ spans the graph, the Euler tour will visit every vertex at least once, but possibly multiple times.

<br>
This is close to a TSP solution, but: 

- it visits each edge twice

- $(d,f)$ should have an edge

- The weight of the Euler tour is equal to twice the MST weight (since we visit each edge twice).


How can we convert this to a proper solution to TSP?

**idea**: 

Compute DFS order, but when we find a repeated edge, instead find the next unvisited vertex.

<center>
    <img src="figures/tsp3.jpg"/>
</center>

The red edges above are called *shortcut edges*.

Because of triangle inequality, we know that the shortcut edges are no longer than the paths they replace
  
  - e.g,. $w(f,c) \le w(\langle f,e,b,a,c \rangle)$
  
  
  
Since we know:

  - $weight(TSP) < weight(MST)$ , and
  
  - $weight(Euler) = 2 \cdot weight(MST)$
  
then we know

 - $weight(MST) \le weight(TSP) \le 2 \cdot weight(MST)$
 